# Clustering Zooniverse Marks to count Iguanas
Count all the iguanas in the images by clustering the marks from the zooniverse volunteers.
This does not compare the results to the gold standard and requires only a single file, the flattened zooniverse data export.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
This notebook is used to cluster the marks from the zooniverse volunteers to count the iguanas in the images saved in results/<phase_tag>/flat_dataset_filtered_Iguanas 3rd launch.csv

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

## Load the data

In [2]:
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold



# Phase Selection
# phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = 3 # None or a number, filter records which have less than these user interactions.

use_gold_standard_subset = None # Use no filtering



## Input Path of all the data
# input_path =Path("/Users/christian/data/zooniverse")
input_path = Path("results/")
# Location for the analysis Results
# output_path = Path(input_path.joinpath(f"2024_03_19_analysis").joinpath(phase_tag))
output_path = Path(input_path.joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)
reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Location for plots
output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)


## Look into the config
This Config points to all files necessary for the analysis + the result files

In [3]:
from zooniverse.config import get_config_all

config = get_config_all(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('results/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': None,
 'gold_standard_image_subset': None,
 'image_source': None,
 'yes_no_dataset': PosixPath('results/Iguanas 3rd launch/yes_no_dataset_Iguanas 3rd launch.csv'),
 'flat_dataset': PosixPath('results/Iguanas 3rd launch/flat_dataset_Iguanas 3rd launch.csv'),
 'merged_dataset': PosixPath('results/Iguanas 3rd launch/flat_dataset_filtered_Iguanas 3rd launch.csv'),
 'comparison_dataset': PosixPath('results/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv'),
 'method_sums': PosixPath('results/Iguanas 3rd launch/Iguanas 3rd launch_method_sums.csv'),
 'dbscan_hyperparam_grid': PosixPath('results/Iguanas 3rd launch/Iguanas 3rd launch_hyperparam_grid.csv')}

In [4]:
from zooniverse.utils.anonymize import UserAnonymizer
from zooniverse.utils.data_format import data_prep_all

if reprocess:
    ds_stats = data_prep_all(phase_tag=phase_tag, 
                         output_path=output_path, 
                         input_path=input_path, 
                         config=config)
    print(ds_stats)

    # Anomymise the data to prevent usernames and user_ids to become public
    anonymizer = UserAnonymizer(config["flat_dataset"])
    anonymizer.anonymize_data()
    anonymizer.save_anonymized_data(config["flat_dataset"])
    
    anonymizer = UserAnonymizer(config["merged_dataset"])
    anonymizer.anonymize_data()
    anonymizer.save_anonymized_data(config["merged_dataset"])

In [5]:
if plot_diagrams == False:
    output_plot_path = None

# the flattened, filtered marks from zooniverse.
df_merged_dataset = pd.read_csv(config["merged_dataset"])



## Look at the data


In [6]:
## Look at the data
df_merged_dataset


,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
0,57,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,301.422882,51.112278,"Others (females, young males, juveniles)",Iguanas 3rd launch,bbe0564f6fa09817cda58d4c1027735e,01bc9deeac290307915eb93166e02a6b
1,58,CaboIbebetsonS,PCIS01-5_67.jpg,78961972,35.903500,468.096375,"Others (females, young males, juveniles)",Iguanas 3rd launch,bbe0564f6fa09817cda58d4c1027735e,01bc9deeac290307915eb93166e02a6b
2,70,WestCoastB,GWB01-3_152.jpg,78925551,728.559448,181.467453,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,a73c4349e66510b82334d70f668ea617
3,71,WestCoastB,GWB01-3_152.jpg,78925551,601.206055,277.385895,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,a73c4349e66510b82334d70f668ea617
4,125,SouthCoastH,ESCH02-1_323.jpg,78965007,247.383331,56.599998,Adult Male not in a lek,Iguanas 3rd launch,59ab2166efbc163b3edd511475247309,ef54f61bd82b88e105449d751a3391b2
...,...,...,...,...,...,...,...,...,...,...
7396,104634,SouthCoastH,ESCH02-1_174.jpg,78964907,688.849609,135.619003,Adult Male with a lek,Iguanas 3rd launch,c17e059e2c3b375b261e417e5a65091d,ae406cf22cbb563674f4b697272a6171
7397,113862,GEB02,GEB02-3_197.jpg,78922625,496.962006,433.519104,Adult Male not in a lek,Iguanas 3rd launch,NaN,25a9604d96ff47b78fa4644b9814a422
7398,113863,GEB02,GEB02-3_197.jpg,78922625,496.542480,416.344574,Adult Male not in a lek,Iguanas 3rd launch,NaN,91a41fd7793b3ec859725897e31e3a5a
7399,113866,GEB02,GEB02-3_197.jpg,78922625,502.057251,430.122284,"Others (females, young males, juveniles)",Iguanas 3rd launch,68a0c0b65edf786c52bd62d8fbdf8c12,0919e52eb9dbee91efcc356faf04d3e6


### Filter User if necessary and Marks


In [7]:
print(f"Before filtering: {df_merged_dataset.subject_id.nunique()}")
# There images in which some people said there are iguanas, but then didn't mark them. Clustering with fewer than 3 dots doesn't make sense
if user_threshold is not None:
    print(f"filtering records which have less than {user_threshold} interactions.")
    df_merged_dataset = filter_df_user_threshold(df_merged_dataset, user_threshold=user_threshold)
    
    
from zooniverse.utils.filters import filter_remove_marks
# Check if partials are still in the data. There shouldn't be any
df_merged_dataset = filter_remove_marks(df_merged_dataset)


2024-03-22 20:53:24.809 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image ESCH02-1_258.jpg has only 1 users, which is less than the threshold of 3
2024-03-22 20:53:24.855 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:25 - filtered out 1 images
2024-03-22 20:53:24.864 | WARNING  | zooniverse.utils.filters:filter_remove_marks:38 - removed 0 partial marks
2024-03-22 20:53:24.866 | WARNING  | zooniverse.utils.filters:filter_remove_marks:39 - After filter_func 86 images are left


Before filtering: 87
filtering records which have less than 3 interactions.


### Are there anonymous users in the data?
There should be some.

In [8]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
1804,20416,SouthCoastH,ESCH01-1_13.jpg,78964714,842.693787,272.662140,Adult Male with a lek,Iguanas 3rd launch,NaN,adc99ea22219d2baff677763af1cd90f
1805,20417,SouthCoastH,ESCH01-1_13.jpg,78964714,890.732117,242.415756,Adult Male with a lek,Iguanas 3rd launch,NaN,adc99ea22219d2baff677763af1cd90f
1806,20418,SouthCoastH,ESCH01-1_13.jpg,78964714,792.876221,124.988678,Adult Male with a lek,Iguanas 3rd launch,NaN,adc99ea22219d2baff677763af1cd90f
1807,20419,SouthCoastH,ESCH01-1_13.jpg,78964714,796.434631,290.454102,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,adc99ea22219d2baff677763af1cd90f
1808,20420,SouthCoastH,ESCH01-1_13.jpg,78964714,816.005798,235.298965,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,adc99ea22219d2baff677763af1cd90f
...,...,...,...,...,...,...,...,...,...,...
1077,11744,WestCoast,PWC03-2-1_42.jpg,78963297,468.903534,196.045349,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,3952ec0af313d58d488dc97beb19c4cb
1479,15812,WestCoast,PWC03-2-1_42.jpg,78963297,431.552338,42.428822,Adult Male with a lek,Iguanas 3rd launch,NaN,d51caf2b0d8a68548a70de2a954d48dd
1480,15813,WestCoast,PWC03-2-1_42.jpg,78963297,465.189301,190.431534,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,d51caf2b0d8a68548a70de2a954d48dd
1481,15814,WestCoast,PWC03-2-1_42.jpg,78963297,365.623840,152.758118,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,d51caf2b0d8a68548a70de2a954d48dd


In [9]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

86

In [10]:
## After filtering there
df_merged_dataset

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
408,4658,SouthCoastH,ESCH01-1_13.jpg,78964714,764.540100,361.271637,Adult Male with a lek,Iguanas 3rd launch,823e0c8bf213199beae68750e41a837a,4253d1b3d5ae39006bb949e6cf2f144e
409,4659,SouthCoastH,ESCH01-1_13.jpg,78964714,767.268921,334.892792,"Others (females, young males, juveniles)",Iguanas 3rd launch,823e0c8bf213199beae68750e41a837a,4253d1b3d5ae39006bb949e6cf2f144e
410,4660,SouthCoastH,ESCH01-1_13.jpg,78964714,769.088196,310.787964,"Others (females, young males, juveniles)",Iguanas 3rd launch,823e0c8bf213199beae68750e41a837a,4253d1b3d5ae39006bb949e6cf2f144e
411,4661,SouthCoastH,ESCH01-1_13.jpg,78964714,754.079529,277.132202,"Others (females, young males, juveniles)",Iguanas 3rd launch,823e0c8bf213199beae68750e41a837a,4253d1b3d5ae39006bb949e6cf2f144e
412,4662,SouthCoastH,ESCH01-1_13.jpg,78964714,808.201660,272.584106,"Others (females, young males, juveniles)",Iguanas 3rd launch,823e0c8bf213199beae68750e41a837a,4253d1b3d5ae39006bb949e6cf2f144e
...,...,...,...,...,...,...,...,...,...,...
5019,64823,WestCoast,PWC03-2-1_42.jpg,78963297,475.094360,192.540146,"Others (females, young males, juveniles)",Iguanas 3rd launch,3fc49e9436026f5e3c1361265a845d75,a1b06094359d2185d0bcee38da83928a
5724,73986,WestCoast,PWC03-2-1_42.jpg,78963297,466.367310,195.423187,"Others (females, young males, juveniles)",Iguanas 3rd launch,c11a32c827347926881e5e1db75cb701,691500ccebe2131f83809524df652f87
5725,73987,WestCoast,PWC03-2-1_42.jpg,78963297,356.656067,149.294586,"Others (females, young males, juveniles)",Iguanas 3rd launch,c11a32c827347926881e5e1db75cb701,691500ccebe2131f83809524df652f87
5726,73988,WestCoast,PWC03-2-1_42.jpg,78963297,421.485443,44.570198,"Others (females, young males, juveniles)",Iguanas 3rd launch,c11a32c827347926881e5e1db75cb701,691500ccebe2131f83809524df652f87


In [11]:
# how many marks per user
df_merged_dataset[["user_id", "x"]].groupby("user_id").count().head()

,x
user_id,
013cb4b55188fc660b8fd7f8dbb9bb8f,9
0156eebf62383fedf03616142d065d39,131
01671783e7cd4124074ed5fc29647828,13
01f32084acb4138c447e1404d765d7b4,2
023383cf6fd2b03c328c2e8054d2ccea,1


In [12]:
# How many images are left in the zooniverse dataset?
df_merged_dataset["subject_id"].nunique()

86

## Clustering

### Basic Statics like mean, median, mode

In [13]:
from zooniverse.analysis import get_mark_overview

basic_stats = []

for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])

    ### basic statistics like mean, median
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    
df_comparison = df_basic_stats


In [14]:
df_basic_stats

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,ESCH01-1_13.jpg,18.0,16.46,19,24,395,"[3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,..."
1,ESCH01-1_19.jpg,6.5,6.39,7,18,115,"[3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, ..."
2,ESCH01-1_21.jpg,6.5,5.96,2,28,167,"[2, 2, 2, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, ..."
3,ESCH01-1_22.jpg,4.0,3.89,5,27,105,"[1, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, ..."
4,ESCH01-1_23.jpg,2.0,2.11,2,27,57,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
81,PWC01-1_61.jpg,14.0,13.25,14,24,318,"[2, 7, 9, 9, 11, 12, 13, 13, 13, 14, 14, 14, 1..."
82,PWC01-1_62.jpg,12.0,11.94,13,16,191,"[10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 1..."
83,PWC01-4_231.jpg,3.0,2.88,1,8,23,"[1, 1, 1, 3, 3, 4, 5, 5]"
84,PWC01-5_34.jpg,1.0,1.46,1,13,19,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3]"


In [15]:
# There might be records with too few annotations if they were not removed before
df_comparison[(df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count


In [16]:
# Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


In [17]:
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,ESCH01-1_13.jpg,18.0,16.46,19,24,395,"[3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,..."
1,ESCH01-1_19.jpg,6.5,6.39,7,18,115,"[3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, ..."
2,ESCH01-1_21.jpg,6.5,5.96,2,28,167,"[2, 2, 2, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, ..."
3,ESCH01-1_22.jpg,4.0,3.89,5,27,105,"[1, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, ..."
4,ESCH01-1_23.jpg,2.0,2.11,2,27,57,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
81,PWC01-1_61.jpg,14.0,13.25,14,24,318,"[2, 7, 9, 9, 11, 12, 13, 13, 13, 14, 14, 14, 1..."
82,PWC01-1_62.jpg,12.0,11.94,13,16,191,"[10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 1..."
83,PWC01-4_231.jpg,3.0,2.88,1,8,23,"[1, 1, 1, 3, 3, 4, 5, 5]"
84,PWC01-5_34.jpg,1.0,1.46,1,13,19,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3]"


In [18]:
df_comparison.sum()

image_name               ESCH01-1_13.jpgESCH01-1_19.jpgESCH01-1_21.jpgE...
median_count                                                         314.5
mean_count                                                          380.51
mode_count                                                             302
users                                                                 1607
sum_annotations_count                                                 7400
annotations_count        [3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,...
dtype: object

### DBSCAN clustering and take the variant with the best silouette score for each image


In [19]:
### The old variant
# from zooniverse.analysis import compare_dbscan_hyp_v2
# 
# eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# min_samples_variants = [3, 5, 8, 10]
# if debug:
#     eps_variants = [0.3]
#     min_samples_variants = [3]
# params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]
# 
# db_scan_results = {}
# db_scan_best_results = []
# db_scan_best_bic_results = []
# for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
# 
#     dbscan_localization = compare_dbscan_hyp_v2(
#         # phase_tag=phase_tag,
#         params=params,
#         df_flat=df_image_name,
#         # output_path=output_path,
#         output_plot_path=output_plot_path,
#         plot=show_plots,
#         
#     )
# 
#     db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
#     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_silouette_score", ascending=False).iloc[0])
#     
# df_dbscan_localization = pd.concat([*db_scan_results.values()])
# df_scan_best_results = pd.DataFrame(db_scan_best_results)



In [20]:
# df_scan_best_results

In [21]:
## fixes the problem with the silouette score sorting
from zooniverse.analysis import compare_dbscan_hyp_v2

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
db_scan_best_bic_results = []
for image_name, df_image_name in df_merged_dataset.groupby("image_name"):

    dbscan_localization = compare_dbscan_hyp_v2(
        params=params,
        df_flat=df_image_name,
        output_plot_path=output_plot_path,
        plot=show_plots
    )

    db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
    
    # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
    # Sillouette Scoring needs a minimum of 2 clusters
    # if there are points in decent radius they will belong to a cluster
    if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        # If two or more cluster seem to exists take ones with the best Silouette score
    else:  
        # take the best result by silouette score if there are more clusters then 1
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)


/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_44322/2583623965.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dbscan_localization = pd.concat([*db_scan_results.values()])


In [22]:
df_scan_best_results

,dbscan_count,dbscan_noise,dbscan_silouette_score,image_name,eps,min_samples,dbscan_BIC_score
10,18,28,0.616352,ESCH01-1_13.jpg,0.10,8,NaN
4,9,12,0.716935,ESCH01-1_19.jpg,0.05,3,NaN
20,8,1,0.861200,ESCH01-1_21.jpg,0.40,3,NaN
20,4,1,0.931680,ESCH01-1_22.jpg,0.40,3,NaN
12,2,1,0.874491,ESCH01-1_23.jpg,0.20,3,NaN
...,...,...,...,...,...,...,...
20,4,3,0.664809,PWC01-1_61.jpg,0.40,3,NaN
12,7,1,0.806874,PWC01-1_62.jpg,0.20,3,NaN
24,4,3,0.752832,PWC01-4_231.jpg,0.50,3,NaN
4,2,8,0.158432,PWC01-5_34.jpg,0.05,3,NaN


Here it can be seen why the silouette score is difficult because it is often undefined.

In [23]:
## save the combinations of parameters, which maximized the silouette score.

df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])
df_scan_best_results

,dbscan_count,dbscan_noise,dbscan_silouette_score,image_name,eps,min_samples,dbscan_BIC_score
10,18,28,0.616352,ESCH01-1_13.jpg,0.10,8,NaN
4,9,12,0.716935,ESCH01-1_19.jpg,0.05,3,NaN
20,8,1,0.861200,ESCH01-1_21.jpg,0.40,3,NaN
20,4,1,0.931680,ESCH01-1_22.jpg,0.40,3,NaN
12,2,1,0.874491,ESCH01-1_23.jpg,0.20,3,NaN
...,...,...,...,...,...,...,...
20,4,3,0.664809,PWC01-1_61.jpg,0.40,3,NaN
12,7,1,0.806874,PWC01-1_62.jpg,0.20,3,NaN
24,4,3,0.752832,PWC01-4_231.jpg,0.50,3,NaN
4,2,8,0.158432,PWC01-5_34.jpg,0.05,3,NaN


In [24]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil" }, inplace=True)

df_comparison = df_comparison.merge(df_scan_best_results, on='image_name', how='left')

In [25]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil
0,ESCH01-1_13.jpg,18.0,16.46,19,24,395,"[3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,...",18
1,ESCH01-1_19.jpg,6.5,6.39,7,18,115,"[3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, ...",9
2,ESCH01-1_21.jpg,6.5,5.96,2,28,167,"[2, 2, 2, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, ...",8
3,ESCH01-1_22.jpg,4.0,3.89,5,27,105,"[1, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, ...",4
4,ESCH01-1_23.jpg,2.0,2.11,2,27,57,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",2
...,...,...,...,...,...,...,...,...
81,PWC01-1_61.jpg,14.0,13.25,14,24,318,"[2, 7, 9, 9, 11, 12, 13, 13, 13, 14, 14, 14, 1...",4
82,PWC01-1_62.jpg,12.0,11.94,13,16,191,"[10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 1...",7
83,PWC01-4_231.jpg,3.0,2.88,1,8,23,"[1, 1, 1, 3, 3, 4, 5, 5]",4
84,PWC01-5_34.jpg,1.0,1.46,1,13,19,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3]",2


### HDBSCAN clustering for each image


In [26]:
from zooniverse.analysis import HDBSCAN_Wrapper

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = HDBSCAN_Wrapper(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [27]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

,image_name,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,ESCH01-1_13.jpg,19,0.0,5,None,26
0,ESCH01-1_19.jpg,9,0.0,5,None,8
0,ESCH01-1_21.jpg,10,0.0,5,None,5
0,ESCH01-1_22.jpg,5,0.0,5,None,6
0,ESCH01-1_23.jpg,2,0.0,5,None,0
...,...,...,...,...,...,...
0,PWC01-1_61.jpg,14,0.0,5,None,36
0,PWC01-1_62.jpg,11,0.0,5,None,5
0,PWC01-4_231.jpg,3,0.0,5,None,0
0,PWC01-5_34.jpg,1,0.0,5,None,12


In [28]:
df_comparison = df_comparison.merge(df_hdbscan, on='image_name', how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,ESCH01-1_13.jpg,18.0,16.46,19,24,395,"[3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,...",18,19,0.0,5,0,26
1,ESCH01-1_19.jpg,6.5,6.39,7,18,115,"[3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, ...",9,9,0.0,5,0,8
2,ESCH01-1_21.jpg,6.5,5.96,2,28,167,"[2, 2, 2, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, ...",8,10,0.0,5,0,5
3,ESCH01-1_22.jpg,4.0,3.89,5,27,105,"[1, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, ...",4,5,0.0,5,0,6
4,ESCH01-1_23.jpg,2.0,2.11,2,27,57,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",2,2,0.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,PWC01-1_61.jpg,14.0,13.25,14,24,318,"[2, 7, 9, 9, 11, 12, 13, 13, 13, 14, 14, 14, 1...",4,14,0.0,5,0,36
82,PWC01-1_62.jpg,12.0,11.94,13,16,191,"[10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 1...",7,11,0.0,5,0,5
83,PWC01-4_231.jpg,3.0,2.88,1,8,23,"[1, 1, 1, 3, 3, 4, 5, 5]",4,3,0.0,5,0,0
84,PWC01-5_34.jpg,1.0,1.46,1,13,19,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3]",2,1,0.0,5,0,12


In [29]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

saved results/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


# A look into the results

## The sum of the clustering
What is the sum of the methods

In [30]:

df_comparison_sum = df_comparison[["median_count", "mean_count", "mode_count", "dbscan_count_sil", "HDBSCAN_count"]].sum().sort_values()
df_comparison_sum.to_csv(config["method_sums"])


In [31]:
print(f"phase_tag: {phase_tag}, user_threshold: {user_threshold}")

phase_tag: Iguanas 3rd launch, user_threshold: 3


## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [32]:
print(f"{config['method_sums'].name}")
df_method_sums = pd.read_csv(config["method_sums"])
df_method_sums

Iguanas 3rd launch_method_sums.csv


,Unnamed: 0,0
0,mode_count,302.00
1,median_count,314.50
2,dbscan_count_sil,333.00
3,HDBSCAN_count,358.00
4,mean_count,380.51


### Comparison per Image Level

In [33]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load results/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


,Unnamed: 0,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,ESCH01-1_13.jpg,18.0,16.46,19,24,395,"[3, 8, 11, 15, 15, 15, 15, 16, 17, 17, 18, 18,...",18,19,0.0,5,0,26
1,1,ESCH01-1_19.jpg,6.5,6.39,7,18,115,"[3, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, 7, 8, 8, ...",9,9,0.0,5,0,8
2,2,ESCH01-1_21.jpg,6.5,5.96,2,28,167,"[2, 2, 2, 2, 2, 2, 3, 3, 5, 5, 5, 6, 6, 6, 7, ...",8,10,0.0,5,0,5
3,3,ESCH01-1_22.jpg,4.0,3.89,5,27,105,"[1, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, ...",4,5,0.0,5,0,6
4,4,ESCH01-1_23.jpg,2.0,2.11,2,27,57,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",2,2,0.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,81,PWC01-1_61.jpg,14.0,13.25,14,24,318,"[2, 7, 9, 9, 11, 12, 13, 13, 13, 14, 14, 14, 1...",4,14,0.0,5,0,36
82,82,PWC01-1_62.jpg,12.0,11.94,13,16,191,"[10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 1...",7,11,0.0,5,0,5
83,83,PWC01-4_231.jpg,3.0,2.88,1,8,23,"[1, 1, 1, 3, 3, 4, 5, 5]",4,3,0.0,5,0,0
84,84,PWC01-5_34.jpg,1.0,1.46,1,13,19,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3]",2,1,0.0,5,0,12
